In [1]:
#################################
# filenames
#################################

number = 24

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1.1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(30)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(30)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(1.1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                    # this is not where this block belongs
                                    # nesting it under "else" above made failures generate "[]" and 
                                    #    only appended new songs to song_list, ignoring those already in url_dict
#                                     #driver.close()
#                                     # clean up and append song to list
#                                     my_json[i].pop('section', None)
#                                     my_json[i]['url'] = url
#                                     #f.write('\n'+str(my_json[i])+',')
#                                     song_list.append(my_json[i])

                                # this is where the above block really belongs 
                                #     so song_list gets every song, even if the url is in url_dict
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1.1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(30)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(30)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1.1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(30)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(30)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1.1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:100])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615163222.558494
500
1/500 5/5,1,465,L5
histo1 1
2/500 6,543/6,D443,664
histo1 1
3/500 16,1,164,242
histo1 1
4/500 5,6,5/6,4
histo1 15
5/500 56/6,642,56/5,5/5
histo1 1
6/500 56,6,5,3
histo1 2
7/500 242,17,1,56
histo1 1
8/500 5/6,6,1,4
histo1 14
9/500 27,b165,4,1
histo1 1
10/500 542,37,6,27
histo1 1
11/500 67,16,4,5
histo1 4
12/500 3,47,365,6
histo1 1
13/500 57/6,D4,D47,D76
histo1 1
14/500 b7,4,b6,5
histo1 1
15/500 67,17,5,4
histo1 1
16/500 5,164,3,4
histo1 1
17/500 57/4,4,542,16
histo1 1
18/500 27,5/6,47,5
histo1 2
19/500 1,5,L26,5/5
histo1 1
20/500 5,6,16,564
histo1 1
21/500 665,M542,643,27
histo1 1
22/500 47,37,b37,b6
histo1 1
23/500 27,37,364,37
histo1 1
24/500 57,77,242,6
histo1 1
25/500 6,1,5/5,27
histo1 1
26/500 5/6,6,642,77/5
histo1 1
27/500 2,1,16,b16
histo1 1
28/500 3,1,6,5
histo1 2
29/500 b4,b7,5/2,5
histo1 1
30/500 6,66,5/6,564/6
histo1 1
31/500 57,1,5,36
histo1 1
32/500 3,6,164,5/5
histo1 1
33/500 17,M17,2,27
histo1 1
34/500 37,47,77,57/6
histo1 1
35/500 6,16,4,2
histo1 1


histo1 1
289/500 5,542,37,27
histo1 1
290/500 642,77/5,4,5
histo1 1
291/500 5,b7,4,67
histo1 1
292/500 564,1,364,6
histo1 1
293/500 3,2,665,57/3
histo1 1
294/500 b7,1,5/5,b7
histo1 1
295/500 1,17,57/4,443
histo1 1
296/500 5/2,17,47,5/2
histo1 1
297/500 6,2,5/6,4
histo1 6
298/500 265,6,265,6
histo1 1
299/500 b7,M1,b7,M1
histo1 1
300/500 57,1,16,4
histo1 10
301/500 4,2,1,L7
histo1 1
302/500 27,67,56,1
histo1 2
303/500 265,67,6,47
histo1 1
304/500 17,57/3,57/6,67
histo1 2
305/500 7,1,464,1
histo1 1
306/500 5/5,464,564,6
histo1 1
307/500 16,6,2,164
histo1 1
308/500 27,1,6,4
histo1 2
309/500 642,4,27,442
histo1 2
310/500 6,5,4,5/6
histo1 30
311/500 56,464,164,4
histo1 1
312/500 3,5/5,1,L7
histo1 3
313/500 16,4,b6,b7
histo1 2
314/500 47,b7,1,27
histo1 1
315/500 16,4,16,1
histo1 8
316/500 1,6,27,5
histo1 13
317/500 6,56,464,564
histo1 2
318/500 1,M17,4,1
histo1 3
319/500 4,464,442,4
histo1 1
320/500 5,56,1,57/4
histo1 1
321/500 5,6,56,66
histo1 4
322/500 1,57,7/6,67
histo1 1
323/500 464,57/4,

In [ ]:
##########################
# list errors here
##########################
246/500 2,4,1,5
2. Unable to extract for 2 4 1 5 page 2
histo1 105
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [2]:
##########################
# test errors here
##########################
mykey = '2,4,1,5'
page = 2
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [2]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 1
# print(histo[:100])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 363, 66, 26, 10, 3, 4, 4, 2, 2, 3, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
